<a href="https://colab.research.google.com/github/KevinCantrell/pHstrips/blob/main/Whatman_3_pad_image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/KevinCantrell/pHstrips.git

Cloning into 'pHstrips'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 178.19 KiB | 7.75 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
import math
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import rcParams
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import pHstrips.data_analysis_helpers as da
import pHstrips.image_processing_source_file as ip

In [ ]:
linLUTfloat=np.zeros((256),dtype='float32')
linLUTint=np.zeros((256),dtype='uint8')
linLUTabs=np.zeros((256),dtype='float32')
for chan in range(256):
    val=chan/255.0
    if (val<=0.04045):
        val=val/12.92
    else:
        val=((val+0.055)/1.055)**2.4
    linLUTfloat[chan]=val
    linLUTint[chan]=int(round(val*255))
    if val==0:
        linLUTabs[chan]=255/64.0
    else:
        linLUTabs[chan]=-np.log10(val)

In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  name=fn
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
settings_file_path="/content/pHstrips/Whatman_3Pad_comRef.set"
settingsFile = open(settings_file_path,'r')
settingString=settingsFile.read()
settingsFile.close()
dictSet=eval(settingString)
print(name)

Saving 2025_08_29_0002.JPG to 2025_08_29_0002 (1).JPG
User uploaded file "2025_08_29_0002 (1).JPG" with length 2780072 bytes
2025_08_29_0002 (1).JPG


In [10]:
cap = cv2.VideoCapture(name)
ret, originalFrame = cap.read()
frame = np.copy(originalFrame)
totalFrames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

totalIndex=int(totalFrames/dictSet['set fr'][0])
standardSwatchStats=np.zeros((16,2,totalIndex+dictSet['set fr'][0],42))
parameterStats=np.zeros((32,8,totalIndex+dictSet['set fr'][0],5))
if totalFrames==1:
    grabbedStats=np.zeros((32,8,100,5))
else:
    grabbedStats=np.zeros((32,8,totalIndex,5))
grabCount=0